# V. Cooper python version of A. Pendergrass script

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
# import xesmf as xe
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import copy
import pandas as pd
import cmocean as cmo
from cartopy.util import add_cyclic_point
import seaborn as sns
%matplotlib inline

import xesmf as xe

In [2]:
# File with the changes in climate: (ts, temp) (TS,T,Q)
varlist2d = ['ts','gw','FSNS', 'FSDS', 'FSNT']
changefile=xr.open_dataset('./demodata/changefields.nc')[varlist2d]
changefile3d=xr.open_dataset('./changefields.plev.nc')
basefile=xr.open_dataset('./demodata/basefields.nc')[varlist2d]
basefile3d=xr.open_dataset('./basefields.plev.nc')

###################

## Read air temperature kernel 
# ta_kernel_hybrid=ncread('kernels/t.kernel.nc','FLNT');
ta_kernel=xr.open_dataset('t.kernel.plev.nc')

## VTC add section to read in pressure levels
p_Pa=ta_kernel.plev
p_hPa=ta_kernel.lev_p 

## this must be generated by calcdp_plev.ncl script
pdiff=xr.open_dataset('dp_plev.nc').dp/100

# p=repmat(permute(repmat(p_hPa,[1 12]),[3 4 1 2]),[288 192 1 1]);
p = p_hPa.values[np.newaxis, :, np.newaxis, np.newaxis] * np.ones(ta_kernel.FLNT.shape)

In [3]:
###################

## Read in coordinate info
lat=xr.open_dataset('./kernels/PS.nc').lat
lon=xr.open_dataset('./kernels/PS.nc').lon
gw=xr.open_dataset('./kernels/t.kernel.nc').gw ## Gaussian weights for CESMgrid
# lev=ncread('kernels/t.kernel.nc','lev'); ## dont need

## Make an area weighting matrix
weight=np.tile(gw.values[:,np.newaxis], len(lon))
weight=weight/np.nansum(weight)
# print(weight.sum())

## Read surface temperature change
dts=changefile.ts

## Calculate the change in global mean surface temperature
dts_globalmean= (dts * weight).sum(dim=('lat','lon')).mean(dim='time')
print('Global mean dTS: ', dts_globalmean.values)
                 

Global mean dTS:  -0.5839361181344885


In [4]:
## Temperature feedback calculation

## Read TOA Longwave surface temperature kernel
ts_kernel=xr.open_dataset('./kernels/ts.kernel.nc').FLNT

## Multiply monthly mean TS change by the TS kernels (function of
## lat, lon, month) (units W/m2)
dLW_ts=ts_kernel *dts
#dLW_ts.mean(dim='time').plot()

## Read air temperature change [lon,lat,level,month]
dta=changefile3d.temp


In [5]:
## Non-pressure level version:
## Read midpoint pressure for each grid cell (lat,lon,level,month), [Pa]
## VTC adjusted this above to be pressure levels
## p=ncread('p_sigma.nc','pmid')/100; %[hPa] 

In [6]:
## Crude tropopause estimate: 100 hPa in the tropics, lowering with
## cosine to 300 hPa at the poles.
x=np.cos(np.deg2rad(lat))
p_tropopause_zonalmean=300-200*x
## VTC
##p_tropopause= ...
##    repmat(permute(repmat(permute(repmat(p_tropopause_zonalmean', ...
##                                         [length(lon) 1]),[1 2 3]),[1 ...
##                    1 length(lev)]),[1 2 3 4]),[1 1 1 12]);
p_tropopause_zonalmean
p_tropopause = p_tropopause_zonalmean.values[
    np.newaxis, np.newaxis,:, np.newaxis] * np.ones(ta_kernel.FLNT.shape)
p_tropopause = xr.DataArray(p_tropopause,dims=changefile3d.dims, coords = changefile3d.coords)

In [7]:
## Set the temperature change to zero in the stratosphere (mask out stratosphere)
dta=xr.where(p>=p_tropopause, dta, np.nan)

## Convolve air temperature kernel with air temperature change
## VTC
## dLW_ta=squeeze(sum(ta_kernel.*dta,3));
dLW_ta=ta_kernel.FLNT * dta.values * pdiff.values

In [8]:
## Add the surface and air temperature response; Take the annual
## average and global area average 
dLW_t_globalmean = -(
    (dLW_ta.sum(dim='plev') + dLW_ts).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
t_feedback=dLW_t_globalmean / dts_globalmean

print('Temperature feedback: ', str(t_feedback.values), ' W m^-2 K^-1')

Temperature feedback:  -3.7459334281195757  W m^-2 K^-1


# Albedo Feedback

In [9]:
## Collect surface shortwave radiation fields to calculate albedo.
## Alternatively, you might already have the change in albedo - that
## would work too.
SW_sfc_net_1=basefile.FSNS
SW_sfc_down_1=basefile.FSDS
SW_sfc_net_2=changefile.FSNS+SW_sfc_net_1
SW_sfc_down_2=changefile.FSDS+SW_sfc_down_1

alb1= 1 - SW_sfc_net_1/SW_sfc_down_1
alb1 = xr.where(np.isnan(alb1), 0, alb1)

alb2= 1 - SW_sfc_net_2/SW_sfc_down_2
alb2 = xr.where(np.isnan(alb2), 0, alb2)

dalb=(alb2-alb1)*100

alb_kernel = xr.open_dataset('./kernels/alb.kernel.nc').FSNT
dSW_alb = alb_kernel * dalb


In [10]:
## average and global area average 
dSW_alb_globalmean = (dSW_alb.mean(dim='time') * weight).sum()
alb_feedback = dSW_alb_globalmean / dts_globalmean

print('Surf. albedo feedback: ', str(alb_feedback.values), ' W m^-2 K^-1')

Surf. albedo feedback:  0.22072796549182414  W m^-2 K^-1


# Water vapor feedback

In [11]:
## Calculate the change in moisture per degree warming at constant relative humidity. 
q1=basefile3d.Q
t1=basefile3d.temp

In [12]:
## addpath scripts/
## VTC script for
def calcsatspechum(t_in, p_in):
    ## T is temperature, P is pressure in hPa 

    ## Formulae from Buck (1981):
    es = (1.0007+(3.46e-6 * p_in)) * 6.1121 * (
        np.exp(17.502*(t_in - 273.15) / (240.97+(t_in - 273.15))))
    
    wsl = 0.622 * es / (p_in - es) ## saturation mixing ratio wrt liquid water (g/kg)
    
    es = (1.0003 + ( 4.18e-6 * p_in)) * 6.1115 * (
        np.exp(22.452 * (t_in - 273.15) / (272.55 + (t_in - 273.15))))
    
    wsi = 0.622 * es / (p - es) ## saturation mixing ratio wrt ice (g/kg)
    
    ws = wsl
    
    ws = xr.where(t_in < 273.15, wsi, ws)
    
    qs = ws / (1+ws) ## saturation specific humidity, g/kg
    return(qs)


In [13]:
qs1 = calcsatspechum(t1,p) #g/kg
qs2 = calcsatspechum(t1+dta,p) #g/kg
dqsdt = (qs2 - qs1) / dta
rh = q1 / qs1
dqdt = rh * dqsdt

## Read kernels
q_LW_kernel = xr.open_dataset('q.kernel.plev.nc').FLNT
q_SW_kernel = xr.open_dataset('q.kernel.plev.nc').FSNT

## Normalize kernels by the change in moisture for 1 K warming at
## constant RH
q_LW_kernel = q_LW_kernel.values / dqdt;
q_SW_kernel = q_SW_kernel.values / dqdt;

## Read the change in moisture
dq = changefile3d.Q

In [14]:
## Set the moisture change to zero in the stratosphere (mask out stratosphere)
dq = xr.where(p>=p_tropopause, dq, np.nan)

In [15]:
## Convolve moisture kernel with change in moisture
dLW_q = q_LW_kernel * dq.values * pdiff.values
dSW_q = q_SW_kernel * dq.values * pdiff.values

In [16]:
## Add the LW and SW responses. Note the sign convention difference
## between LW and SW!
dR_q_globalmean = (
    (-dLW_q + dSW_q).sum(dim='lev_p').mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
q_feedback = dR_q_globalmean / dts_globalmean

print('Water vapor feedback: ', str(q_feedback.values), ' W m^-2 K^-1')



Water vapor feedback:  1.6206921057233556  W m^-2 K^-1


# WV with log of Q

In [17]:
## Calculate the change in moisture per degree warming at constant relative humidity. 
## Run the accompanying NCL script with your input files, or
## implement here.                                                             

In [18]:
q0 = q1 ## AP used q1 for this above #basefile3d.Q #kg/kg 

## all of these are set above
#t1 = basefile3d.temp set above
#dta = changefile3d.temp ## set above
#qs1 = calcsatspechum(t1,p); % g/kg
#qs2 = calcsatspechum(t1+dta,p); % g/kg
#dqsdt = (qs2 - qs1)./dta;

## slightly different from above
rh = 1000*q0 / qs1
dqdt = rh * dqsdt ## assume constant RH
dlogqdt= dqdt / (1000 * q0) ## convert denominator to g/kg

## Re-read kernels, 
## normalize by the change in moisture for 1 K warming at
## constant RH using log Q
q_LW_kernel = xr.open_dataset('q.kernel.plev.nc').FLNT.values
logq_LW_kernel = q_LW_kernel / dlogqdt
q_SW_kernel = xr.open_dataset('q.kernel.plev.nc').FSNT.values
logq_SW_kernel = q_SW_kernel / dlogqdt

In [19]:
## all set above
## Read the change in moisture
## dq=ncread(changefile3d,'Q');
## Mask out the stratosphere
##dq=dq.*(p>=p_tropopause);

dlogq = dq / q0


In [20]:
## Convolve moisture kernel with change in moisture
dLW_logq = logq_LW_kernel.values * dlogq * pdiff.values
dSW_logq = logq_SW_kernel.values * dlogq * pdiff.values

## Add the LW and SW responses. 
## Note the sign convention difference LW and SW
dR_logq_globalmean = (
    (-dLW_logq + dSW_logq).sum(dim='lev_p').mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
logq_feedback = dR_logq_globalmean / dts_globalmean

print('logQ WV feedback: ', str(logq_feedback.values), ' W m^-2 K^-1')
print('linQ WV feedback: ', str(q_feedback.values), ' W m^-2 K^-1')

logQ WV feedback:  1.6206921057233559  W m^-2 K^-1
linQ WV feedback:  1.6206921057233556  W m^-2 K^-1


# Planck Feedback

In [21]:
## Project surface temperature change into height 
##VTC
##dts3d=repmat(permute(dts,[1 2 4 3]),[1 1 30 1]);
dts3d = dts + changefile3d.temp-changefile3d.temp

## Mask stratosphere
dt_planck = xr.where(p>=p_tropopause, dts3d, np.nan)


In [22]:
## Convolve air temperature kernel with 3-d surface air temp change
##VTC
##dLW_planck=squeeze(sum(ta_kernel_hybrid.*dt_planck,3));
# dLW_planck = squeeze(sum(ta_kernel * dt_planck.*pdiff,3))
dLW_planck=ta_kernel.FLNT * dt_planck.values * pdiff.values

## Take the annual average and global area average; incorporate the
## part due to surface temperature change itself 
dLW_planck_globalmean = -(
    (dLW_planck.sum(dim='plev') + dLW_ts).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
planck_feedback=dLW_planck_globalmean / dts_globalmean


In [23]:
## Lapse rate feedback                                                                                                                                                                 
## Calculate the departure of temperature change from the surface
## temperature change
dt_lapserate=xr.where(p>=p_tropopause, dta-dt_planck, np.nan)

## Convolve air temperature kernel with 3-d surface air temp change
## VTC
## dLW_lapserate=squeeze(sum(ta_kernel.*dt_lapserate,3));
# dLW_lapserate=squeeze(sum(ta_kernel.*dt_lapserate.*pdiff,3));
dLW_lapserate = ta_kernel.FLNT * dt_lapserate.values * pdiff.values

## Take the annual average and global area average 
dLW_lapserate_globalmean = -(
    (dLW_lapserate.sum(dim='plev')).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
lapserate_feedback = dLW_lapserate_globalmean / dts_globalmean

print('Planck feedback: ', 
      str(planck_feedback.values), ' W m^-2 K^-1')
print('Lapse rate feedback: ', 
      str(lapserate_feedback.values), ' W m^-2 K^-1')

### SANITY CHECK: Do the Planck and lapse-rate feedbacks add up to
### the total temperature feedback? (They should)

## Planck + lapse rate feedback
total_t_feedback = planck_feedback+lapserate_feedback

print('Temperature feedback: ',
      str(t_feedback.values), ' W m^-2 K^-1')
print('Planck+lapse rate components: ',
      str(total_t_feedback.values), ' W m^-2 K^-1')


Planck feedback:  -3.261498255128057  W m^-2 K^-1
Lapse rate feedback:  -0.48443517299151895  W m^-2 K^-1
Temperature feedback:  -3.7459334281195757  W m^-2 K^-1
Planck+lapse rate components:  -3.7459334281195757  W m^-2 K^-1


## Cloud Feedback

In [24]:
## STEP 1. Calculate total-sky and clear-sky feedbacks
lev = ta_kernel.plev.values ## this is in Pa

In [26]:
## Read TOA Longwave surface temperature kernel
ts_kernel_clearsky = xr.open_dataset('./kernels/ts.kernel.nc').FLNTC

## Multiply monthly mean TS change by the TS kernels 
## (function of lat, lon, month) (units W/m2)
dLW_ts_cs = ts_kernel_clearsky * dts

## Read clear-sky air temperature kernel
ta_kernel_clearsky = xr.open_dataset('./t.kernel.plev.nc').FLNTC

## Convolve air temperature kernel with air temperature change
dLW_ta = dLW_ta.sum(dim='plev')
dLW_ta_cs = (ta_kernel_clearsky.values * dta).sum(dim='lev_p')

In [27]:
## ALBEDO clear sky

## Read TOA albedo kernel
alb_kernel_clearsky = xr.open_dataset('./kernels/alb.kernel.nc').FSNTC

dSW_alb_cs = alb_kernel_clearsky.values * dalb

In [28]:
## WATER VAPOR clear sky

## read kernels
q_LW_kernel_clearsky = xr.open_dataset('./q.kernel.plev.nc').FLNTC
q_SW_kernel_clearsky = xr.open_dataset('./q.kernel.plev.nc').FSNTC

## Normalize kernels by the change in moisture for 1 K warming at
## constant RH (linear)
rh = q1 / qs1
dqdt = rh * dqsdt ## from above, reset to linear method
q_LW_kernel_clearsky = q_LW_kernel_clearsky.values / dqdt
q_SW_kernel_clearsky = q_SW_kernel_clearsky.values / dqdt

## Convolve moisture kernel with change in moisture
dLW_q = dLW_q.sum(dim='lev_p')
dSW_q = dSW_q.sum(dim='lev_p')
dLW_q_cs = (q_LW_kernel_clearsky.values * dq).sum(dim='lev_p')
dSW_q_cs = (q_SW_kernel_clearsky.values * dq).sum(dim='lev_p')

In [29]:
### Change in Cloud Radiative Effect (CRE) 
d_sw = changefile3d.FSNT
d_sw_cs = changefile3d.FSNTC
d_lw = changefile3d.FLNT
d_lw_cs = changefile3d.FLNTC

d_cre_sw = d_sw_cs - d_sw
d_cre_lw = d_lw_cs - d_lw


In [30]:
### Cloud masking of radiative forcing
ghgfile = './forcing/ghg.forcing.nc'
sw = xr.open_dataset(ghgfile).FSNT
sw_cs = xr.open_dataset(ghgfile).FSNTC
lw = xr.open_dataset(ghgfile).FLNT
lw_cs = xr.open_dataset(ghgfile).FLNTC
ghg_sw = sw_cs-sw
ghg_lw = lw_cs-lw

aerosolfile = './forcing/aerosol.forcing.nc';
sw = xr.open_dataset(aerosolfile).FSNT
sw_cs = xr.open_dataset(aerosolfile).FSNTC
lw = xr.open_dataset(aerosolfile).FLNT
lw_cs = xr.open_dataset(aerosolfile).FLNTC
aerosol_sw = sw_cs - sw
aerosol_lw = lw_cs - lw

cloud_masking_of_forcing_sw = aerosol_sw + ghg_sw
cloud_masking_of_forcing_lw = aerosol_lw + ghg_lw


In [31]:
### Cloud feedback
### CRE + cloud masking of radiative forcing + corrections for each feedback

dLW_cloud = -d_cre_lw+cloud_masking_of_forcing_lw + (
    dLW_q_cs - dLW_q.values) + (dLW_ta_cs - dLW_ta.values)+(dLW_ts_cs-dLW_ts)
dSW_cloud = -d_cre_sw+cloud_masking_of_forcing_sw + (
    dSW_q_cs - dSW_q.values) + (dSW_alb_cs-dSW_alb)


In [33]:
## Take global and annual averages
dLW_cloud_globalmean = (-dLW_cloud.mean(dim='time') * weight).sum()
dSW_cloud_globalmean = ( dSW_cloud.mean(dim='time') * weight).sum()

## Divide by global, annual mean temperature change to get W/m2/K
lw_cloud_feedback = dLW_cloud_globalmean / dts_globalmean
sw_cloud_feedback = dSW_cloud_globalmean / dts_globalmean

print('LW Cloud feedback: ',
      str(lw_cloud_feedback.values), ' W m^-2 K^-1')
print('SW Cloud feedback: ',
      str(sw_cloud_feedback.values), ' W m^-2 K^-1')

LW Cloud feedback:  -0.7736452846514574  W m^-2 K^-1
SW Cloud feedback:  0.7161558311211548  W m^-2 K^-1


# All Feedbacks

In [35]:
print('Planck feedback: ', 
      str(planck_feedback.values), ' W m^-2 K^-1')
print('Lapse rate feedback: ', 
      str(lapserate_feedback.values), ' W m^-2 K^-1')
print('Water vapor feedback: ', 
      str(q_feedback.values), ' W m^-2 K^-1')
print('Surf. albedo feedback: ', 
      str(alb_feedback.values), ' W m^-2 K^-1')
print('LW Cloud feedback: ',
      str(lw_cloud_feedback.values), ' W m^-2 K^-1')
print('SW Cloud feedback: ',
      str(sw_cloud_feedback.values), ' W m^-2 K^-1')

sumall = planck_feedback + lapserate_feedback + q_feedback + alb_feedback + (
    lw_cloud_feedback + sw_cloud_feedback)
print('\n\nSum all feedbacks: ',
      str(sumall.values), ' W m^-2 K^-1')

## extras
print('\n\nTemperature feedback: ',
      str(t_feedback.values), ' W m^-2 K^-1')
print('Planck+lapse rate components: ',
      str(total_t_feedback.values), ' W m^-2 K^-1')

print('\n\nlogQ WV feedback: ', str(logq_feedback.values), ' W m^-2 K^-1')
print('linQ WV feedback: ', str(q_feedback.values), ' W m^-2 K^-1')

Planck feedback:  -3.261498255128057  W m^-2 K^-1
Lapse rate feedback:  -0.48443517299151895  W m^-2 K^-1
Water vapor feedback:  1.6206921057233556  W m^-2 K^-1
Surf. albedo feedback:  0.22072796549182414  W m^-2 K^-1
LW Cloud feedback:  -0.7736452846514574  W m^-2 K^-1
SW Cloud feedback:  0.7161558311211548  W m^-2 K^-1


Sum all feedbacks:  -1.9620028104346985  W m^-2 K^-1


Temperature feedback:  -3.7459334281195757  W m^-2 K^-1
Planck+lapse rate components:  -3.7459334281195757  W m^-2 K^-1


logQ WV feedback:  1.6206921057233559  W m^-2 K^-1
linQ WV feedback:  1.6206921057233556  W m^-2 K^-1
